# Explorer la base de données de scénarios AR6 du GIEC
2026-02-05 A. Danneaux

*codé avec pas mal de Claude Opus 4.5*

## Qu'est-ce que la base de données AR6 ?

La **base de données AR6** est une collection complète de scénarios d'atténuation du changement climatique compilée pour le Sixième Rapport d'Évaluation (AR6) du GIEC. Elle contient des données provenant de divers **Modèles d'Évaluation Intégrée (IAM)** qui simulent les trajectoires futures des systèmes énergétiques, des émissions et des résultats climatiques.

### Composants clés de la base de données :

| Composant | Description |
|-----------|-------------|
| **Modèles** | Différents IAMs (ex: IMAGE, REMIND, MESSAGE) qui simulent les systèmes énergie-économie-climat |
| **Scénarios** | Différentes hypothèses sur les politiques, technologies et développement socio-économique |
| **Variables** | Indicateurs comme les émissions de CO2, la consommation d'énergie, les investissements, les capacités |
| **Régions** | Zones géographiques (Monde, régions R10, pays individuels) |
| **Périodes** | Généralement 2020-2100 par pas de 5 ou 10 ans |

### Catégories de scénarios (C1-C8) :

Les scénarios sont classés selon leurs résultats en termes de température :

| Catégorie | Description | Objectif de température |
|----------|-------------|-------------------|
| **C1** | Limiter le réchauffement à 1,5°C avec dépassement nul/limité | <1,5°C |
| **C2** | Limiter le réchauffement à 1,5°C avec dépassement important | <1,5°C (dépassement) |
| **C3** | Limiter probablement le réchauffement à 2°C | <2°C (>67% de probabilité) |
| **C4** | Limiter le réchauffement à 2°C | <2°C (>50% de probabilité) |
| **C5** | Limiter le réchauffement à 2,5°C | <2,5°C |
| **C6** | Limiter le réchauffement à 3°C | <3°C |
| **C7** | Limiter le réchauffement à 4°C | <4°C |
| **C8** | Dépasser 4°C | >4°C |

### Trajectoires Illustratives d'Atténuation (IMPs) :

Le GIEC met en avant des scénarios spécifiques comme **Trajectoires Illustratives d'Atténuation** pour représenter différentes stratégies :

| IMP | Nom | Description | Modèle | Scénario dans la base |
|-----|------|-------------|--------|----------------------|
| **CurPol** | Politiques Actuelles | Continuation des politiques actuelles | GCAM 5.3 | NGFS2_Current Policies |
| **ModAct** | Action Modérée | Ambition modérée type CDN | IMAGE 3.0 | EN_INDCi2030_3000f |
| **Neg** | Émissions Négatives | Forte dépendance au retrait de carbone | COFFEE 1.1 | EN_NPi2020_400f_lowBECCS |
| **Ren** | Fort Renouvelable | Déploiement rapide des énergies renouvelables | REMIND-MAgPIE | DeepElec_SSP2_ HighRE_Budg900 |
| **LD** | Faible Demande | Réduction de la demande par l'efficacité | MESSAGEix-GLOBIOM | LowEnergyDemand_1.3_IPCC |
| **GS** | Renforcement Graduel | Renforcement progressif des politiques | WITCH 5.0 | CO_Bridge |
| **SP** | Trajectoires Durables | Trajectoire de développement durable | REMIND-MAgPIE | SusDev_SDP-PkBudg1000 |



## Partie A : Chargement et Exploration des Données

### Étape 1 : Importer les bibliothèques nécessaires

Nous avons besoin des bibliothèques Python suivantes :
- `pandas` : Pour la manipulation de données
- `numpy` : Pour les opérations numériques  
- `matplotlib` : Pour créer des visualisations
- `pyam` : Une bibliothèque spécialisée pour travailler avec les données de scénarios IAM

In [ ]:
# Importer les bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyam
from ar6_plotting_tools import plot_database_composition

# Paramètres d'affichage pour une meilleure visualisation
pd.set_option('display.max_columns', 30)
plt.style.use('seaborn-v0_8-whitegrid')

print("Bibliothèques importées avec succès !")

### Étape 2 : Charger les données depuis la base IIASA

Nous utilisons `pyam.read_iiasa()` pour télécharger les données directement depuis la base IIASA.

**Paramètres :**
- `'ar6-public'` : Le nom de la base AR6 publique
- `region` : La ou les région(s) géographique(s) à charger
- `variable` : La ou les variable(s) à télécharger

⚠️ **Note :** La première exécution peut prendre quelques minutes pour télécharger les données.

In [ ]:
import os

# Définir les variables à télécharger
Variables = [
    'Emissions|CO2|Energy and Industrial Processes',  # Émissions de CO2 de l'énergie et l'industrie
    'Primary Energy|Coal',                             # Énergie primaire du charbon
    'Primary Energy|Oil',                              # Énergie primaire du pétrole
    'Primary Energy|Gas',                              # Énergie primaire du gaz
    'Primary Energy|Solar',                            # Énergie primaire solaire
    'Primary Energy|Wind',                             # Énergie primaire éolienne
    "Primary Energy|Biomass",
    "Primary Energy|Nuclear",
    'Investment|Energy Supply|Electricity|Solar',      # Investissement dans l'électricité solaire
    'Investment|Energy Supply|Electricity|Wind',       # Investissement dans l'électricité éolienne
    "Investment|Energy Supply|Extraction|Fossil",
    "Investment|Energy Supply|Electricity|Transmission and Distribution",
    'AR6 climate diagnostics|Surface Temperature (GSAT)|FaIRv1.6.2|50.0th Percentile',
    "Secondary Energy|Electricity",
    "Capacity|Electricity|Gas",
    "Capacity|Electricity|Solar",
    # Variables de l'équation de Kaya (pour la section bonus)
    'GDP|PPP',                                         # PIB en parité de pouvoir d'achat
    'Population',                                       # Population
    'Primary Energy',                                   # Énergie primaire totale

]

# Définir la ou les région(s) à télécharger
Regions = ['World']

# Vérifier si le fichier existe déjà
if os.path.exists('ar6_scenarios_data.xlsx'):
    print("Fichier 'ar6_scenarios_data.xlsx' trouvé, chargement des données locales...")
    df_pyam = pyam.IamDataFrame('ar6_scenarios_data.xlsx')
    print("Données chargées avec succès !")
else:

    # Télécharger les données depuis la base IIASA
    print("Téléchargement des données depuis IIASA... Cela peut prendre quelques minutes.")
    df_pyam = pyam.read_iiasa('ar6-public', region=Regions, variable=Variables)
    print("Données téléchargées avec succès !")

### Étape 3 : Afficher et explorer le DataFrame

Comprenons la structure de nos données. La bibliothèque `pyam` fournit un objet spécial `IamDataFrame` avec des méthodes utiles pour travailler avec les données de scénarios.

In [ ]:
# Afficher les informations de base sur les données
print("="*60)
print("📊 APERÇU DES DONNÉES")
print("="*60)
print(f"\nNombre de modèles : {len(df_pyam.model)}")
print(f"Nombre de scénarios : {len(df_pyam.scenario)}")
print(f"Nombre de variables : {len(df_pyam.variable)}")
print(f"Régions : {df_pyam.region}")
print(f"Période : {min(df_pyam.year)} - {max(df_pyam.year)}")

In [ ]:
# Convertir en DataFrame pandas timeseries pour une manipulation plus facile
df = df_pyam.timeseries()

# Afficher les premières lignes
print("📊 10 premières lignes des données de séries temporelles :")
print("="*80)
df.head(10)

### Explication des valeurs NaN (Not a Number)

Les valeurs **NaN** dans les données de séries temporelles sont normales et proviennent de plusieurs sources :

| Raison | Explication |
|--------|-------------|
| **Pas de temps différents** | Certains modèles rapportent des données tous les 5 ans (2020, 2025, 2030...), d'autres tous les 10 ans, créant des NaN pour les années intermédiaires |
| **Horizons temporels variables** | Tous les scénarios ne couvrent pas la même période (certains commencent en 2005, d'autres en 2020) |
| **Variables non simulées** | Certains modèles ne calculent pas toutes les variables (ex: capacités électriques, investissements) |
| **Données historiques manquantes** | Les années avant 2005-2010 peuvent être absentes selon les modèles |


In [ ]:
# Vérifier la structure du multi-index
print("📋 Niveaux d'index (lignes) :")
print(df.index.names)
print("\n📋 Colonnes (années) :")
print(list(df.columns))

### Étape 4 : Obtenir les catégories de scénarios (Métadonnées)

Les catégories de scénarios (C1-C8) sont stockées dans les **métadonnées** de l'objet pyam. Extrayons-les.

In [ ]:
# Obtenir les métadonnées (inclut les catégories de scénarios)
meta = df_pyam.meta

# Afficher les colonnes de métadonnées
print("📋 Colonnes de métadonnées disponibles :")
print(meta.columns.tolist())

# Afficher les premières lignes des métadonnées
print("\n📊 Échantillon des métadonnées :")
meta.head()

In [ ]:
# Compter les scénarios par catégorie
if 'Category' in meta.columns:
    category_col = 'Category'
elif 'category' in meta.columns:
    category_col = 'category'
else:
    # Trouver la colonne contenant l'information de catégorie
    category_col = [col for col in meta.columns if 'categ' in col.lower()][0] if any('categ' in col.lower() for col in meta.columns) else None

if category_col:
    print(f"📊 Nombre de scénarios par catégorie (colonne : '{category_col}') :")
    print(meta[category_col].value_counts().sort_index())
else:
    print("⚠️ Colonne de catégorie non trouvée dans les métadonnées")

In [ ]:
# Importer les fonctions pour analyser la composition de la base
from ar6_plotting_tools import (
    assign_model_families,
    assign_assessment_status,
    assign_ssp_family,
    count_scenarios_by_group,
    plot_scenario_counts,
)

### Étape 5 : Composition de la base - Familles de modèles et scénarios SSP

Visualisons **quels modèles et scénarios** sont représentés dans la base.


In [ ]:
# Enrichir les métadonnées
meta_enriched = assign_model_families(df_pyam.meta)
meta_enriched = assign_assessment_status(meta_enriched)
meta_enriched = assign_ssp_family(meta_enriched)

# Compter par famille de modèles et par famille SSP
model_counts = count_scenarios_by_group(meta_enriched, 'Model_Family', n_top=12)
ssp_counts = count_scenarios_by_group(meta_enriched, 'SSP_Family')

# Visualiser côte à côte
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
plot_scenario_counts(model_counts, ax=axes[0], title='Par famille de modèles')
plot_scenario_counts(ssp_counts, ax=axes[1], title='Par famille SSP')
fig.tight_layout();

**Graph a) reproduced from source :** Peters, Glen P., Alaa Al Khourdajie, Ida Sognnaes, and Benjamin M. Sanderson. ‘AR6 Scenarios Database: An Assessment of Current Practices and Future Recommendations’. Npj Climate Action 2, no. 1 (2023): 1. https://doi.org/10.1038/s44168-023-00050-9.

---
## Partie B : Visualisation des Données

### Définir les couleurs et fonctions d'aide

Nous allons définir un code couleur pour chaque catégorie de scénario afin de rendre nos graphiques visuellement cohérents et faciles à interpréter.

In [ ]:
# Importer les outils de visualisation (fonctions dans un fichier séparé)
from ar6_plotting_tools import (
    plot_timeseries_by_category, 
    plot_scatter_boxplot_by_category,
    plot_kaya_decomposition,
    plot_kaya_variable,
    plot_kaya_ratio,
    create_kaya_explorer,
    show_color_palette,
    show_imp_palette,
    CATEGORY_COLORS,
    ALL_CATEGORIES,
    IMP_SCENARIOS,
    IMP_DETAILS,
)

In [ ]:
# Vérifier quels IMPs sont disponibles dans nos données
# (Les IMPs doivent être dans les données téléchargées pour les variables sélectionnées)
print("Vérification des Trajectoires Illustratives d'Atténuation dans nos données :")
print("="*60)

df_ts = df_pyam.timeseries()
for imp_name, scenario_name in IMP_SCENARIOS.items():
    model_name = IMP_DETAILS[imp_name][0]
    
    # Rechercher cet IMP dans les données
    found = False
    for idx in df_ts.index:
        if idx[0] == model_name and idx[1] == scenario_name:
            found = True
            break
    
    status = "Trouvé" if found else "Non trouvé"
    print(f"{status}: {imp_name} ({model_name} / {scenario_name})")

### B.1) Graphique de séries temporelles : Code couleur par catégorie

Ce graphique montre des séries temporelles où chaque ligne représente un scénario, coloré selon sa catégorie (C1-C8).

**Comment utiliser :**
- Modifiez le paramètre `variable` pour explorer différentes métriques
- Utilisez `categories` pour sélectionner les catégories à afficher (ex: `categories=['C1', 'C2', 'C3']`)
- Utilisez `show_imps=True` pour mettre en évidence les **Trajectoires Illustratives d'Atténuation**
- Ajustez `alpha` (transparence) s'il y a beaucoup de lignes superposées

#### 🟠 Essayez vous-même ! 

Modifiez la variable dans la cellule ci-dessous pour explorer différentes métriques. 

**Variables disponibles :**
- `'Emissions|CO2|Energy and Industrial Processes'`
- `'Primary Energy|Coal'`
- `'Primary Energy|Oil'`
- `'Primary Energy|Gas'`
- `'Primary Energy|Solar'`
- `'Primary Energy|Wind'`
- `'Investment|Energy Supply|Electricity|Solar'`
- `'Investment|Energy Supply|Electricity|Wind'`

**Options :**
```python
# Afficher seulement certaines catégories
plot_timeseries_by_category(df_pyam, variable, categories=['C1', 'C2', 'C3'])

# Mettre en évidence les Trajectoires Illustratives d'Atténuation (IMPs)
plot_timeseries_by_category(df_pyam, variable, show_imps=True)
```

In [ ]:
variable = 'AR6 climate diagnostics|Surface Temperature (GSAT)|FaIRv1.6.2|50.0th Percentile'
plot_timeseries_by_category(df_pyam, variable, alpha=0.4, categories = ALL_CATEGORIES, show_imps=True);

In [ ]:
# ============================================
# 🟠 MODIFIEZ LA VARIABLE ICI POUR EXPLORER !
# ============================================


variable = 'Emissions|CO2|Energy and Industrial Processes'  # <-- Essayez de changer !

# Tracer toutes les catégories (par défaut)
plot_timeseries_by_category(df_pyam, variable, alpha=0.4, categories = ALL_CATEGORIES, show_imps=True);

In [ ]:
# Tracer avec les Trajectoires Illustratives d'Atténuation (IMPs) mises en évidence
# Les IMPs sont affichées avec des lignes colorées et un contour noir par-dessus tous les scénarios
plot_timeseries_by_category(df_pyam, 'Primary Energy|Coal', 
                            show_imps=True, alpha=0.3, categories=['C1','C7']);

---
### B.2) Nuage de points et Boxplot à une date choisie

Cette visualisation montre la distribution des valeurs à une **année spécifique** selon les différentes catégories de scénarios. 

- **Points** : Valeurs individuelles des scénarios
- **Boxplot** : Statistiques résumées (médiane, quartiles, étendue)

**Comment utiliser :**
- Modifiez `variable` et `year` pour explorer différentes combinaisons
- Utilisez `categories` pour sélectionner les catégories à afficher

#### 🟠 Essayez vous-même !

Modifiez les paramètres `variable` et `year` pour explorer différentes combinaisons.

**Années suggérées à explorer :**
- `2030` : Court terme (année cible des CDN)
- `2050` : Mi-siècle (année cible de neutralité carbone pour de nombreux pays)
- `2100` : Fin de siècle

**Optionnel :** Utilisez `categories` pour filtrer :
```python
# Comparer seulement les scénarios extrêmes
plot_scatter_boxplot_by_category(df_pyam, variable, year, categories=['C1', 'C8'])
```

In [ ]:
# ============================================
# 🟠 MODIFIEZ LA VARIABLE ET L'ANNÉE ICI !
# ============================================

variable = 'Emissions|CO2|Energy and Industrial Processes'  # <-- Essayez de changer !
year = 2100  # <-- Essayez : 2030, 2050, ou 2100

# Tracer le nuage de points/boxplot
plot_scatter_boxplot_by_category(df_pyam, variable, year, show_imps =True);

In [ ]:
# Exemple : Comparer les émissions en 2030 vs 2100 (côte à côte)
# Ici nous créons notre propre figure et passons les axes à la fonction

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

variable = 'Emissions|CO2|Energy and Industrial Processes'

# Tracer sur le premier axe (2030)
plot_scatter_boxplot_by_category(df_pyam, variable, 2030, ax=axes[0])
axes[0].set_title(f'Émissions de CO2 en 2030')

# Tracer sur le second axe (2100)
plot_scatter_boxplot_by_category(df_pyam, variable, 2100, ax=axes[1])
axes[1].set_title(f'Émissions de CO2 en 2100')

[ax.set_ylim(
    [min([axs.get_ylim()[0] for axs in axes]), max([axs.get_ylim()[1] for axs in axes])]
) for ax in axes]


fig.tight_layout()


In [ ]:
# Exemple : Regarder les investissements solaires en 2050 (seulement pour les scénarios ambitieux)
plot_scatter_boxplot_by_category(df_pyam, 'Investment|Energy Supply|Electricity|Solar', 2050,
                                  categories=['C1', 'C2', 'C3', 'C8']);

In [ ]:

plot_scatter_boxplot_by_category(df_pyam, "Investment|Energy Supply|Extraction|Fossil", 2050,
                                  categories=['C1',  'C3','C5','C7','C8']);

---
## Bonus : L'identité de Kaya

### Comprendre les déterminants des émissions de CO₂

L'**identité de Kaya** décompose les émissions de CO₂ en quatre facteurs clés :

$$\text{CO}_2 = \underbrace{P}_{\text{Population}} \times \underbrace{\frac{PIB}{P}}_{\text{PIB par habitant}} \times \underbrace{\frac{E}{PIB}}_{\text{Intensité énergétique}} \times \underbrace{\frac{CO_2}{E}}_{\text{Intensité carbone}}$$

Où :
- **P** = Population (milliards d'habitants)
- **PIB/P** = PIB par habitant 
- **E/PIB** = Intensité énergétique de l'économie (combien d'énergie nécessaire par unité de PIB)
- **CO₂/E** = Intensité carbone de l'énergie (combien de CO₂ émis par unité d'énergie)



### Étapes 1 & 2 : Explorateur Interactif de Kaya (Trajectoire individuelle)

Les variables de Kaya (PIB, Population, Énergie Primaire, Émissions de CO₂) ont été chargées au début du tutoriel.

**Utilisez l'outil interactif ci-dessous pour :**
1. **Explorer** une trajectoire : voir ses valeurs de variables de Kaya à n'importe quelle année
2. **Calculer** des ratios personnalisés : construisez votre propre ratio en sélectionnant numérateur et dénominateur

In [ ]:
# Lancer l'explorateur interactif de Kaya (Étapes 1 & 2)
kaya_explorer = create_kaya_explorer(df_pyam)
display(kaya_explorer)

### 📊 Étape 3 : Comparer TOUTES les trajectoires - Graphique de décomposition de Kaya

Maintenant, visualisons la **décomposition complète de Kaya** pour toutes les Trajectoires Illustratives d'Atténuation.

**Fonctions disponibles :**
- `plot_kaya_variable(df_pyam, variable, ...)` - Trace une variable (CO2, Population, etc.)
- `plot_kaya_ratio(df_pyam, numerator, denominator, ...)` - Trace un ratio (PIB/Pop, E/PIB, CO2/E)


#### 🟠 Essayez différents modes :
```python
# Tracer une seule variable
plot_kaya_variable(df_pyam, 'Emissions|CO2|Energy and Industrial Processes', mode='both')

# Tracer un ratio personnalisé
plot_kaya_ratio(df_pyam, numerator='GDP|PPP', denominator='Population', mode='both')

```

In [ ]:
# Créer la figure complète en utilisant les fonctions individuelles
fig, axs = plt.subplots(2, 3, figsize=(10, 7))

# Variables de Kaya
co2_var = 'Emissions|CO2|Energy and Industrial Processes'
pop_var = 'Population'
gdp_var = 'GDP|PPP'
energy_var = 'Primary Energy'

# Rangée 1 : CO2, Population, PIB/habitant
plot_kaya_variable(df_pyam, co2_var, ax=axs[0,0], title='CO2 Emissions', mode='both', categories=['C1','C2','C7'])
plot_kaya_variable(df_pyam, pop_var, ax=axs[0,1], title='Population', mode='both', categories=['C1','C2','C7'])
plot_kaya_ratio(df_pyam, gdp_var, pop_var, ax=axs[0,2], title='GDP per Capita', mode='both', categories=['C1','C2','C7'])

# Rangée 2 : Légende, Intensité énergétique, Intensité carbone
axs[1,0].axis('off')  # Espace pour légende
plot_kaya_ratio(df_pyam, energy_var, gdp_var, ax=axs[1,1], title='Energy Intensity (E/GDP)', mode='both', categories=['C1','C3','C7'])
plot_kaya_ratio(df_pyam, co2_var, energy_var, ax=axs[1,2], title='Carbon Intensity (CO2/E)', mode='both', categories=['C1','C2','C6','C8'])

fig.tight_layout();